### Pymaceuticals Skill Drill - Day 1

Congratulations, you are hired by Pymaceuticals Inc., one of the leading imaginary pharmaceutical companies that specializes in anti-cancer pharmaceuticals, to assist their senior scientist team in the effort to begin screening for potential treatments for squamous cell carcinoma (SCC), a commonly occurring form of skin cancer.

In this study, 249 mice identified with SCC tumor growth were treated through a variety of drug regimens. Over the course of 45 days, tumor development was observed and measured. The purpose of this study was to compare the performance of Pymaceuticals' drug of interest, Capomulin, versus the other treatment regimens. You have been tasked by the senior scientist team to generate an initial drug regimens comparison and generate a summary of your findings. 

For this skill drill, you will walk through the steps of a basic analysis and visualize our dataset using a new type of visualization - a box and whisker plot. Although we have provided all of the steps required to produce each output, there may be some new concepts and/or terminology in this skill drill you may not have seen before. If you are ever stuck or confused, try googling some of the terms or check out the resource links we provide throughout the activity. You got this!

### Data Cleaning 

In [1]:
%matplotlib inline

In [15]:
# Import dependencies
import pandas as pd
import os
import matplotlib.pyplot as plt

In [3]:
# Read the mouse data and the study results
mouse_file = os.path.join('../','Resources', 'Mouse_metadata.csv')
study_file = os.path.join('../','Resources', 'Study_results.csv')
mouse_df = pd.read_csv(mouse_file)
study_df = pd.read_csv(study_file)

In [4]:
# Display the mouse data
mouse_df.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g)
0,k403,Ramicane,Male,21,16
1,s185,Capomulin,Female,3,17
2,x401,Capomulin,Female,16,15
3,m601,Capomulin,Male,22,17
4,g791,Ramicane,Male,11,16


In [5]:
# Display the study data
study_df.tail()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
1888,r944,45,41.581521,2
1889,u364,45,31.023923,3
1890,p438,45,61.433892,1
1891,x773,45,58.634971,4
1892,b879,45,72.555239,2


In [6]:
# Combine the data into a single dataset and display it
merge_df = pd.merge(mouse_df, study_df, how="left", on="Mouse ID")
merge_df

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [7]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
ndms=merge_df.loc[merge_df.duplicated(subset=['Mouse ID', 'Timepoint']),'Mouse ID'].unique()


In [8]:
# Optional: Get all the data for the duplicate mouse ID. 

In [9]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID and display it
data_complete = merge_df[merge_df['Mouse ID'].isin(ndms)==False]
data_complete.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


### Generating the Boxplot

In [10]:
# Determine the final timepoint for each mouse.

# Start by getting the greatest timepoint for each mouse
max_time = data_complete.groupby(['Mouse ID'])['Timepoint'].max()
max_time = max_time.reset_index()
max_time.head()

,Mouse ID,Timepoint
0,a203,45
1,a251,45
2,a262,45
3,a275,45
4,a366,30


In [12]:
# Join the newly created `.max()` dataframe to the dataframe from Part 1
merge2_df = max_time.merge(data_complete, on=['Mouse ID', 'Timepoint'], how='left')
merge2_df.head()

,Mouse ID,Timepoint,Drug Regimen,Sex,Age_months,Weight (g),Tumor Volume (mm3),Metastatic Sites
0,a203,45,Infubinol,Female,20,23,67.973419,2
1,a251,45,Infubinol,Female,21,25,65.525743,1
2,a262,45,Placebo,Female,17,29,70.717621,4
3,a275,45,Ceftamin,Female,20,28,62.999356,3
4,a366,30,Stelasyn,Female,16,29,63.440686,1


In [13]:
# Create a list with all 10 drug regimens.
drug_list = ['Infubinol', 'Placebo', 'Ceftamin', 'Stelasyn', 'Zoniferol',
       'Ramicane', 'Ketapril', 'Propriva', 'Naftisol', 'Capomulin']

# Create a empty list to fill with the tumor vol data
tumor_vol_data = []

# Isolate (filter) each mice on each drug to collect their tumor volume.
for drug in drug_list:

    # Locate the rows which match the drug and get the final tumor volumes of all mice
    final_tumor_vol = merge2_df.loc[merge2_df['Drug Regimen'] == drug]['Tumor Volume (mm3)']
        
    # Append the outcome to the empty list previously created.
    tumor_vol_data.append(final_tumor_vol)

In [21]:
# Create a boxplot that visualizes the final tumor volume of all mice in the study across all drug regimens.

# Define a custom shape for all outliers in the visualization
circle = dict(markerfacecolor='bl', marker='s')

# Create horizontal box and whisker plot
plt.boxplot(tumor_vol_data, labels = drug_list, flierprops=circle, vert=False)
plt.xlabel('Final Tumor Volume (mm3)')
plt.show()

ValueError: Invalid RGBA argument: 'bl'

<Figure size 432x288 with 1 Axes>